# Prequential RoBERTa DPR Pool

Within the paper [WatClaimCheck: A new Dataset for Claim Entailment and Inference](https://aclanthology.org/2022.acl-long.92.pdf), they state that the Dense Passage Retrieval model using a pooling.

## Prequential

The prequential dataset are partitions of 6 months such that it simulates how new topics are treated given new information. (See Section 5.2.1 of the paper)

## How do they pool?
1. For each claim, all of the sentences are pooled from every associated premise article and ranked using a similarity score.
2. Evidence sentences are concatenated in the descending order of similarity score.
3. The claim text and evidence sentences are concatenated.
4. Resulting text is truncated to maximum sequence length of transformer model to perform claim veracity inference

In [1]:
from helper import download_prequential_dataset
from transformers import RobertaConfig, RobertaTokenizer, TFRobertaForSequenceClassification

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

2023-10-24 21:11:44.679340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 21:11:45.468361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set Global Variables
DATASET_FP = "./WatClaimCheck_dataset" # CHANGE TO MATCH LOCAL

In [28]:
# Retrieve dataset
full_pd_df = download_prequential_dataset(DATASET_FP)[0]

Extracting fields from metadata
Extracting fields from label


In [34]:
len(full_pd_df)

33721

In [31]:
full_pd_df['review_date'].isnull().sum()


654

In [33]:
full_pd_df['claim_date'].isnull().sum()

9740

In [35]:
len(full_pd_df[(full_pd_df['review_date'].isnull()) & (full_pd_df['claim_date'].isnull())])

0

In [18]:
# Set model properties
max_seq_len = 512
model_checkpoint = "roberta-base"
learning_rate = 0.00001

In [19]:
# Get model and set number of labels
model_config = RobertaConfig.from_pretrained(model_checkpoint)
model_config.num_labels = 3
model = TFRobertaForSequenceClassification.from_pretrained(model_checkpoint, config=model_config)

# Get Tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

# Get optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti